#### Day 9: Advanced Joins 🚀

Welcome to Day 9! Today, we’ll dive deeper into the world of SQL joins, exploring advanced join techniques to handle more complex data relationships.

#### 1. Database Connection

In [ ]:
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="30_Days_SQL"
)
cursor = conn.cursor()
print("Connected to '30_Days_SQL'!")

---
#### 1. Self Join ✍️
A self join is a join where a table is joined with itself. This is useful for hierarchical data like employee-manager relationships.

In [ ]:
cursor.execute("SET FOREIGN_KEY_CHECKS=0")
cursor.execute("DROP TABLE IF EXISTS employees")
cursor.execute("SET FOREIGN_KEY_CHECKS=1")

cursor.execute('''
CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    name VARCHAR(50),
    age INT,
    salary DECIMAL(10,2),
    department_id INT,
    manager_id INT
)
''')

employees_data = [
    (1, 'Alice', 45, 120000, 1, None),
    (2, 'Bob', 30, 80000, 1, 1),
    (3, 'Charlie', 35, 85000, 1, 1),
    (4, 'David', 25, 60000, 2, 2)
]
cursor.executemany("INSERT INTO employees VALUES (%s, %s, %s, %s, %s, %s)", employees_data)
conn.commit()

query = """
SELECT e1.name AS employee, e2.name AS manager
FROM employees e1
LEFT JOIN employees e2
ON e1.manager_id = e2.employee_id;
"""
pd.read_sql(query, conn)

#### 2. Cross Join 🌱
A cross join returns the Cartesian product of two tables.

In [ ]:
cursor.execute("DROP TABLE IF EXISTS colors")
cursor.execute("DROP TABLE IF EXISTS products")

cursor.execute('''
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(50) NOT NULL,
    category VARCHAR(50),
    price DECIMAL(10, 2),
    stock_quantity INT
)
''')

cursor.execute("CREATE TABLE colors (color_id INT PRIMARY KEY, color_name VARCHAR(50))")

cursor.executemany("INSERT INTO products VALUES (%s, %s, %s, %s, %s)", [
    (1, 'Pen', 'Stationery', 1.50, 500),
    (2, 'Notebook', 'Stationery', 5.00, 200)
])
cursor.executemany("INSERT INTO colors VALUES (%s, %s)", [(1, 'Red'), (2, 'Blue')])
conn.commit()

query = """
SELECT p.product_name, c.color_name
FROM products p
CROSS JOIN colors c;
"""
pd.read_sql(query, conn)

#### 3. Using Joins with Aggregates 🏊
Calculating total sales by customer using JOIN and GROUP BY.

In [ ]:
cursor.execute("SET FOREIGN_KEY_CHECKS=0")
cursor.execute("DROP TABLE IF EXISTS customers")
cursor.execute("DROP TABLE IF EXISTS orders")
cursor.execute("SET FOREIGN_KEY_CHECKS=1")

cursor.execute('''
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(50),
    city VARCHAR(50),
    region VARCHAR(50),
    email VARCHAR(100)
)
''')

cursor.execute('''
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    order_date DATE,
    order_total DECIMAL(10, 2),
    comments TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)
''')

cursor.executemany("INSERT INTO customers VALUES (%s, %s, %s, %s, %s)", [
    (1, 'John', 'New York', 'East', 'john@example.com'),
    (2, 'Sarah', 'London', 'UK', 'sarah@example.com')
])
cursor.executemany("INSERT INTO orders VALUES (%s, %s, %s, %s, %s, %s)", [
    (101, 1, 1, '2024-01-10', 100.00, 'Delivered'),
    (102, 1, 2, '2024-01-15', 150.00, 'Processing'),
    (103, 2, 1, '2024-02-01', 200.00, 'Shipped')
])
conn.commit()

query = """
SELECT c.customer_name, SUM(o.order_total) AS total_sales
FROM customers c
JOIN orders o
ON c.customer_id = o.customer_id
GROUP BY c.customer_name;
"""
pd.read_sql(query, conn)

In [ ]:
conn.close()